In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from textblob import TextBlob


In [3]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("NewsDataInspection").getOrCreate()

# Cargar el archivo CSV
file_path = '../data/raw/news.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

24/08/16 20:38:02 WARN Utils: Your hostname, mauro-thinkpad resolves to a loopback address: 127.0.1.1; using 192.168.0.212 instead (on interface wlp0s20f3)
24/08/16 20:38:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/16 20:38:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Contar el número de filas y columnas
num_rows = df.count()
num_cols = len(df.columns)
print(f"\nDimensiones del DataFrame: {num_rows} filas, {num_cols} columnas")


Dimensiones del DataFrame: 1911 filas, 3 columnas


In [5]:
# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
df.show(5,truncate=False)

Primeras filas del DataFrame:
+----------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
# Mostrar el esquema del DataFrame (tipos de datos de cada columna)
print("\nEsquema del DataFrame:")
df.printSchema()


Esquema del DataFrame:
root
 |-- titulo: string (nullable = true)
 |-- link: string (nullable = true)
 |-- cuerpo: string (nullable = true)



In [7]:
# Describir las estadísticas básicas
print("\nDescripción estadística de las columnas numéricas:")
df.describe().show()



Descripción estadística de las columnas numéricas:


+-------+--------------------+--------------------+--------------------+
|summary|              titulo|                link|              cuerpo|
+-------+--------------------+--------------------+--------------------+
|  count|                1910|                1911|                1906|
|   mean|                NULL|                NULL|                NULL|
| stddev|                NULL|                NULL|                NULL|
|    min|          """Bury Me|           My Love""|    dice su creador"|
|    max|“Tenemos paciente...|https://www.lanac...|“¡Pon a remojar l...|
+-------+--------------------+--------------------+--------------------+



In [8]:
# Verificar si hay valores nulos
print("\nNúmero de valores nulos por columna:")
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()


Número de valores nulos por columna:
+------+----+------+
|titulo|link|cuerpo|
+------+----+------+
|     1|   0|     5|
+------+----+------+



In [9]:
# Filtrar los registros que tienen valores nulos en alguna columna
df_with_nulls = df.filter(F.greatest(*[F.col(c).isNull().cast("int") for c in df.columns]) == 1)

# Mostrar los registros con nulos
print("Registros con valores nulos en alguna columna:")
df_with_nulls.show(truncate=False)

Registros con valores nulos en alguna columna:
+--------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|titulo                                                                    |link                                                                                                                                                            |cuerpo                                      

In [10]:
# Eliminar los registros que tienen algún valor nulo
df_cleaned = df.dropna()

In [11]:
# Filtrar los registros que tienen valores nulos en alguna columna
df_with_nulls = df_cleaned.filter(F.greatest(*[F.col(c).isNull().cast("int") for c in df_cleaned.columns]) == 1)

# Mostrar los registros con nulos
print("Registros con valores nulos en alguna columna:")
df_with_nulls.show(truncate=False)

Registros con valores nulos en alguna columna:
+------+----+------+
|titulo|link|cuerpo|
+------+----+------+
+------+----+------+



In [14]:
# Definir una función para calcular el sentimiento usando TextBlob
def get_sentiment(text):
    if text is None:
        return None
    return TextBlob(text).sentiment.polarity

# Registrar la función como una UDF (User Defined Function)
sentiment_udf = F.udf(get_sentiment)

# Crear la columna 'sentiment' basada en el cuerpo de la noticia
df_with_sentiment = df.withColumn("sentiment", sentiment_udf(F.col("cuerpo")))

# Mostrar algunas filas con la nueva columna de sentimiento
print("Primeras filas con puntuación de sentimiento:")
df_with_sentiment.show(3)

Primeras filas con puntuación de sentimiento:
+--------------------+--------------------+--------------------+-------------------+
|              titulo|                link|              cuerpo|          sentiment|
+--------------------+--------------------+--------------------+-------------------+
|Johanna’s Vision:...|https://www.lanac...|TGA, una empresa ...|0.24444444444444444|
|UFL: el videojueg...|https://www.lanac...|El mundo de los v...| 0.6666666666666666|
|Star Wars Jedi: S...|https://www.lanac...|Electronic Arts (...|                0.6|
+--------------------+--------------------+--------------------+-------------------+
only showing top 3 rows



In [15]:
df_with_sentiment.write.csv("../data/processed/news.csv", header=True)